In [ ]:
import warnings
warnings.simplefilter("ignore")
import gzip
import json
import pandas as pd

pd.set_option('display.max_columns', None)


In [ ]:

def parse_json_gz(file_path):
        with open(file_path, 'r') as f:
            for line in f:
                yield json.loads(line)

reviews = list(parse_json_gz(r'C:\MS DS\RG_Winter25\Capstone 1 DSCI 591\data\review-Pennsylvania.json\review-Pennsylvania.json'))
reviews = pd.DataFrame(reviews)
reviews.drop(columns=['pics'], inplace=True) 



In [ ]:

reviews['time'] = pd.to_datetime(reviews['time'], unit='ms')
reviews = reviews.sort_values(by='time', ascending=True)

object_cols = reviews.select_dtypes(include=['object']).columns
reviews[object_cols] = reviews[object_cols].astype(str)

reviews.drop_duplicates(inplace=True)

def handle_missing_values(df):
    """
    Cleans the dataset by handling missing values:
    - Drops rows where 'user_id', 'text', or 'time' are missing.
    - Fills NaN values in 'text' and 'resp' columns with an empty string.
    """
    drop_subset = ['user_id', 'text', 'time'] 
    text_columns = ['text', 'resp']  
    
    df = df.drop_duplicates(subset=drop_subset)
    df = df.dropna(subset=drop_subset)

    for col in text_columns:
        if col in df.columns:
            df[col] = df[col].fillna("")
    
    return df

reviews["text"] = reviews["text"].str.split("\(Original\)").str[0].str.strip()
        

reviews = handle_missing_values(reviews)

columns_to_fix = reviews[object_cols]
for col in columns_to_fix:
    reviews[col] = reviews[col].astype(str).str.replace("\x00", "", regex=True)

reviews.rename(columns={'name': 'customer_name'}, inplace=True)
reviews = reviews.reset_index(drop=True).rename_axis('id').reset_index()
reviews.head()


,id,user_id,customer_name,time,rating,text,resp,gmap_id
0,0,113859720763765753174,Joe Nicolas,1990-12-31,5.0,None,None,0x89c6c89ada6725cd:0x2479810721359c7
1,1,113513174935774808551,Glenn Vinogradov,1990-12-31,5.0,The best French bistrot outside of France!,None,0x89c6c89e45a8bb15:0x9e89f0fed5054247
2,2,107208310941555070685,Stephen Kossuth,1990-12-31,4.0,None,None,0x89c6f1441b55e805:0xaf705e17955b9e4e
3,3,117887984087038418367,Melody Tsai,1990-12-31,5.0,Great Food!,None,0x89c6c60e85b145bf:0xb189cdb4c35a41f3
4,4,111910324014559174367,Phylis Feiner Johnson,1990-12-31,5.0,Sean is the master of the grill. Imitators bew...,None,0x89c6ed4fea04defb:0x868746dba6d82c78


In [3]:
reviews.shape

(21699421, 8)

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
from concurrent.futures import ThreadPoolExecutor


DB_HOST = "localhost"  
DB_PORT = "5432"  
DB_NAME = "Capstone"  
DB_USER = "postgres"  
DB_PASSWORD = "Google
TABLE_NAME = "reviews"  
CHUNK_SIZE = 400  

engine = create_engine(f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

from tqdm import tqdm

def insert_chunk(chunk, engine, table_name):
    """
    Inserts a chunk of pandas DataFrame rows into a PostgreSQL table.
    """
    try:
        chunk.to_sql(table_name, engine, if_exists="append", index=False)
    except Exception as e:
        print(f"Error inserting chunk: {e}")

def dump_reviews_table(df_reviews):
    """
    Dumps the 'reviews' DataFrame into the PostgreSQL 'reviews' table in chunks,
    with tqdm progress tracking.
    """
    try:
        total_rows = len(df_reviews)
        num_chunks = (total_rows // CHUNK_SIZE) + 1  
        
        for start in tqdm(range(0, total_rows, CHUNK_SIZE), desc="Dumping Data", total=num_chunks):
            chunk = df_reviews.iloc[start: start + CHUNK_SIZE]  
            insert_chunk(chunk, engine, TABLE_NAME)
        
        print(f"Data successfully dumped into table '{TABLE_NAME}' in PostgreSQL!")
    except Exception as e:
        print("Error:", e)

dump_reviews_table(reviews)



NameError: name 'reviews' is not defined

In [6]:
reviews = None

## Dumping Business Data

In [ ]:

def parse_json_gz(file_path, num_lines=300):
    """
    Reads a specified number of lines from a JSON file.
    
    Parameters:
        file_path (str): Path to the JSON file.
        num_lines (int): Number of lines to read.
        
    Yields:
        dict: Parsed JSON object.
    """
    with open(file_path, 'r') as f:
        for i, line in enumerate(f):
            if num_lines:
                if i >= num_lines:
                    break
            yield json.loads(line)

file_path = r'C:\MS DS\RG_Winter25\Capstone 1 DSCI 591\data\meta-Pennsylvania.json\meta-Pennsylvania.json'
meta_data = list(parse_json_gz(file_path, num_lines=None))

meta_data = pd.DataFrame(meta_data)

if 'pics' in meta_data.columns:  
    meta_data.drop(columns=['pics'], inplace=True) 

meta_data.to_csv('./data/sample_meta_data.csv', index=False)
meta_data.rename(columns={'name': 'business_name','MISC':'misc'}, inplace=True)
meta_data = meta_data.drop_duplicates(subset=['gmap_id','longitude','latitude'])
meta_data = meta_data.dropna(subset=['gmap_id','longitude','latitude'])
meta_data.head()

,business_name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,misc,state,relative_results,url
0,Primo Pizza & Subs,"Primo Pizza & Subs, 148 W St Joseph St, Easton...",0x89c46d5e4554eae1:0xa2f8b211524ca29a,None,40.682177,-75.209820,[Pizza delivery],4.5,25,$$,"[[Thursday, 11AM–9PM], [Friday, 11AM–10PM], [S...","{'Service options': ['No-contact delivery', 'D...",Open ⋅ Closes 9PM,"[0x89c46c56cac9c661:0xdc9d0dd3f2b203ae, 0x89c4...",https://www.google.com/maps/place//data=!4m2!3...
1,S.J. O’Hara Inc.,"S.J. O’Hara Inc., 143 White Oak Ct, Tannersvil...",0x89c48c790b767ec7:0x1943c20a42196a68,None,41.055487,-75.284536,"[HVAC contractor, Air conditioning contractor,...",4.5,8,None,"[[Thursday, 7AM–7PM], [Friday, 7AM–7PM], [Satu...","{'Health & safety': ['Appointment required', '...",Open ⋅ Closes 7PM,"[0x89c48c7bd96b49f9:0xdf562ab873031e1, 0x89c45...",https://www.google.com/maps/place//data=!4m2!3...
2,Vivi Bubble Tea,"Vivi Bubble Tea, 701 S 5th St, Philadelphia, P...",0x89c6c89efcaed69d:0xded973f6033e7dba,"Bubble tea shop offering hot & cold versions, ...",39.940293,-75.150923,[Restaurant],4.0,8,None,None,"{'Service options': ['Delivery'], 'Amenities':...",None,"[0x89c6c62a40f4b733:0x73bfeec1898f272b, 0x89c6...",https://www.google.com/maps/place//data=!4m2!3...
3,Kater Street Bicycle,"Kater Street Bicycle, 609 S 16th St, Philadelp...",0x89c6c63c8cd87141:0x54d0d283872eecbb,None,39.943673,-75.168762,"[Bicycle Shop, Bicycle repair shop]",4.9,15,None,"[[Thursday, 11AM–7PM], [Friday, 2–6PM], [Satur...",None,Permanently closed,"[0x89c6c623342d6197:0x70233738cd92d9b, 0x89c6c...",https://www.google.com/maps/place//data=!4m2!3...
4,Cadence Cycling and Cafe - Center City,"Cadence Cycling and Cafe - Center City, 201 S ...",0x89c6c715d1821fe3:0x9cfa8308c0ce2289,None,39.950078,-75.180729,"[Bicycle Shop, Bicycle repair shop, Coffee sho...",4.8,38,None,"[[Thursday, 7AM–7PM], [Friday, 7AM–7PM], [Satu...","{'Service options': ['In-store pickup', 'In-st...",Open ⋅ Closes 7PM,"[0x89c6c7d1a79d4103:0x2476b0fe30eb0871, 0x89c6...",https://www.google.com/maps/place//data=!4m2!3...


In [3]:
meta_data.shape

(189836, 15)

In [ ]:
def safe_convert_list(value):
    """Convert string representation of lists to actual lists."""
    if isinstance(value, str) and value.startswith("[") and value.endswith("]"):
        try:
            return ast.literal_eval(value)  
        except (SyntaxError, ValueError):
            return value  
    return value

def convert_price(value):
    """Map price symbols to numeric values."""
    price_map = {'$': 1, '$$': 2, '$$$': 3}
    return price_map.get(value, None)  

meta_data['category'] = meta_data['category'].apply(safe_convert_list)
meta_data['relative_results'] = meta_data['relative_results'].apply(safe_convert_list)
meta_data['hours'] = meta_data['hours'].apply(safe_convert_list)

meta_data.replace({'None': None}, inplace=True)

meta_data['price'] = meta_data['price'].apply(convert_price)

meta_data = meta_data.where(pd.notna(meta_data), None)

def serialize_misc(data):
    if isinstance(data, dict):
        return json.dumps(data)  
    return data  

meta_data['misc'] = meta_data['misc'].apply(serialize_misc)

In [ ]:

from tqdm import tqdm

TABLE_NAME = 'business'
CHUNK_SIZE = 5000
def dump_reviews_table(df_reviews):
    """
    Dumps the 'reviews' DataFrame into the PostgreSQL 'reviews' table in chunks,
    with tqdm progress tracking.
    """
    try:
        total_rows = len(df_reviews)
        num_chunks = (total_rows // CHUNK_SIZE) + 1  
        
        for start in tqdm(range(0, total_rows, CHUNK_SIZE), desc="Dumping Data", total=num_chunks):
            chunk = df_reviews.iloc[start: start + CHUNK_SIZE]  
            insert_chunk(chunk, engine, TABLE_NAME)
        
        print(f"Data successfully dumped into table '{TABLE_NAME}' in PostgreSQL!")
    except Exception as e:
        print("Error:", e)

dump_reviews_table(meta_data)


Dumping Data:   8%|▊         | 3/38 [00:00<00:02, 11.69it/s]

Error inserting chunk: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "business_pkey"
DETAIL:  Key (gmap_id)=(0x89c46d5e4554eae1:0xa2f8b211524ca29a) already exists.

[SQL: INSERT INTO business (business_name, address, gmap_id, description, latitude, longitude, category, avg_rating, num_of_reviews, price, hours, misc, state, relative_results, url) VALUES (%(business_name__0)s, %(address__0)s, %(gmap_id__0)s, %(descripti ... 290182 characters truncated ... rice__999)s, %(hours__999)s, %(misc__999)s, %(state__999)s, %(relative_results__999)s, %(url__999)s)]
[parameters: {'price__0': 2.0, 'avg_rating__0': 4.5, 'address__0': 'Primo Pizza & Subs, 148 W St Joseph St, Easton, PA 18042', 'business_name__0': 'Primo Pizza & Subs', 'gmap_id__0': '0x89c46d5e4554eae1:0xa2f8b211524ca29a', 'longitude__0': -75.20981979999999, 'description__0': None, 'state__0': 'Open ⋅ Closes 9PM', 'url__0': 'https://www.google.com/maps/place//data=!4m2!3m1!1s0x89c46d5e4554eae1:0xa2f8b

Dumping Data:  13%|█▎        | 5/38 [00:00<00:02, 11.98it/s]

Error inserting chunk: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "business_pkey"
DETAIL:  Key (gmap_id)=(0x8834f4a8dbff5e31:0xb812cd71d4d089ff) already exists.

[SQL: INSERT INTO business (business_name, address, gmap_id, description, latitude, longitude, category, avg_rating, num_of_reviews, price, hours, misc, state, relative_results, url) VALUES (%(business_name__0)s, %(address__0)s, %(gmap_id__0)s, %(descripti ... 290182 characters truncated ... rice__999)s, %(hours__999)s, %(misc__999)s, %(state__999)s, %(relative_results__999)s, %(url__999)s)]
[parameters: {'price__0': None, 'avg_rating__0': 4.4, 'address__0': 'Flowerama Pittsburgh, 3111 Babcock Blvd, Pittsburgh, PA 15237', 'business_name__0': 'Flowerama Pittsburgh', 'gmap_id__0': '0x8834f4a8dbff5e31:0xb812cd71d4d089ff', 'longitude__0': -80.0134933, 'description__0': 'Florist chain offering plants, special arrangements, balloons & gifts as well as flowers & bouquets.', 'state__0': 'Closed ⋅ 

Dumping Data:  24%|██▎       | 9/38 [00:00<00:02, 12.05it/s]

...

Error inserting chunk: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "business_pkey"
DETAIL:  Key (gmap_id)=(0x89c6c87f5fd65fa1:0xc0b681e0f2e52559) already exists.

[SQL: INSERT INTO business (business_name, address, gmap_id, description, latitude, longitude, category, avg_rating, num_of_reviews, price, hours, misc, state, relative_results, url) VALUES (%(business_name__0)s, %(address__0)s, %(gmap_id__0)s, %(descripti ... 290182 characters truncated ... rice__999)s, %(hours__999)s, %(misc__999)s, %(state__999)s, %(relative_results__999)s, %(url__999)s)]
[parameters: {'price__0': 2.0, 'avg_rating__0': 4.6, 'address__0': 'Sazon Restaurant & Cafe, 941 Spring Garden St, Philadelphia, PA 19123', 'business_name__0': 'Sazon Restaurant & Cafe', 'gmap_id__0': '0x89c6c87f5fd65fa1:0xc0b681e0f2e52559', 'longitude__0': -75.154375, 'description__0': 'Arepas & other traditional Venezuelan fare, including several gluten-free options. Cash only.', 'state__0': 'Open ⋅ 

Dumping Data:  92%|█████████▏| 35/38 [00:03<00:00, 10.90it/s]

Error inserting chunk: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "business_pkey"
DETAIL:  Key (gmap_id)=(0x89c510832a876cdf:0x7075beef1f190cd5) already exists.

[SQL: INSERT INTO business (business_name, address, gmap_id, description, latitude, longitude, category, avg_rating, num_of_reviews, price, hours, misc, state, relative_results, url) VALUES (%(business_name__0)s, %(address__0)s, %(gmap_id__0)s, %(descripti ... 290182 characters truncated ... rice__999)s, %(hours__999)s, %(misc__999)s, %(state__999)s, %(relative_results__999)s, %(url__999)s)]
[parameters: {'price__0': 1.0, 'avg_rating__0': 3.8, 'address__0': 'Metro by T-Mobile, 18A Gateway Shopping Center, Edwardsville, PA 18704', 'business_name__0': 'Metro by T-Mobile', 'gmap_id__0': '0x89c510832a876cdf:0x7075beef1f190cd5', 'longitude__0': -75.89985659999999, 'description__0': 'Wireless provider supplying phones, no-contract plans, accessories & repair services.', 'state__0': 'Closed ⋅ Ope

Dumping Data: 100%|██████████| 38/38 [00:03<00:00,  9.62it/s]

Error inserting chunk: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "business_pkey"
DETAIL:  Key (gmap_id)=(0x89c8e11b58bc4bf1:0xfbb007771536b147) already exists.

[SQL: INSERT INTO business (business_name, address, gmap_id, description, latitude, longitude, category, avg_rating, num_of_reviews, price, hours, misc, state, relative_results, url) VALUES (%(business_name__0)s, %(address__0)s, %(gmap_id__0)s, %(descripti ... 290182 characters truncated ... rice__999)s, %(hours__999)s, %(misc__999)s, %(state__999)s, %(relative_results__999)s, %(url__999)s)]
[parameters: {'price__0': None, 'avg_rating__0': 5.0, 'address__0': None, 'business_name__0': 'S&A Maintenance Service LLC', 'gmap_id__0': '0x89c8e11b58bc4bf1:0xfbb007771536b147', 'longitude__0': -77.26111949999999, 'description__0': None, 'state__0': None, 'url__0': 'https://www.google.com/maps/place//data=!4m2!3m1!1s0x89c8e11b58bc4bf1:0xfbb007771536b147?authuser=-1&hl=en&gl=us', 'category__0': ['Hand